This notebook reproduces the fluorescent protein design experiments whose results are shown in Figs. 3, 4, S2.

In [ ]:
import os
import sys
import time
from importlib import reload
module_path = os.path.abspath(os.path.join('../'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
import numpy as np
    
import assay
import calibrate as cal

In [ ]:
reload(cal)
reload(assay)
alpha = 0.1                           # miscoverage
n_trains = [96, 192, 384]             # number of training points
ntrain2reg = {96: 10, 192: 1, 384: 1} # ridge regularization strength (gamma in code and paper)
n_seed = 5000                         # number of random trials
lmbdas = [0, 2, 4, 6]                 # lambda, inverse temperature
y_increment = 0.01                    # grid spacing between candidate label values, \Delta in paper
ys = np.arange(0, 1.7, y_increment)   # candidate label values, \mathcal{Y} in paper
order = 2                             # complexity of features. 1 encodes the AA at each site,
                                      # 2 the AAs at each pair of sites,
                                      # 3 the AAs at each set of 3 sites, etc.
        
# likelihood under training input distribution, p_X in paper (uniform distribution)
ptrain_fn = lambda x: (1.0 / np.power(2, 13)) * np.ones([x.shape[0]])
for fitness_str in ['blue', 'red']:
    
    # featurize all sequences in combinatorially complete dataset
    data = assay.PoelwijkData(fitness_str, order=order)
    
    for t, n_train in enumerate(n_trains):

        reg = ntrain2reg[n_train]
        fcs = cal.ConformalRidgeFeedbackCovariateShift(ptrain_fn, ys, data.X_nxp, reg)
        scs = cal.ConformalRidgeStandardCovariateShift(ptrain_fn, ys, data.X_nxp, reg)

        for l, lmbda in enumerate(lmbdas):

            fset_s, sset_s = [], []
            fcov_s, scov_s = np.zeros([n_seed]), np.zeros([n_seed])
            ytest_s, predtest_s = np.zeros([n_seed]), np.zeros([n_seed])
            t0 = time.time()

            for seed in range(n_seed):
                
                # sample training and designed data
                Xtrain_nxp, ytrain_n, Xtest_1xp, ytest_1, pred_1 = cal.get_training_and_designed_data(
                    data, n_train, reg, lmbda, seed=seed )
                ytest_s[seed] = ytest_1[0]
                predtest_s[seed] = pred_1[0]

                # construct confidence set under feedback covariate shift
                fset, _ = fcs.get_confidence_set(Xtrain_nxp, ytrain_n, Xtest_1xp, lmbda, alpha=alpha) 
                fset_s.append(fset)
                fcov_s[seed] = cal.is_covered(ytest_s[seed], fset, y_increment)

                # construct confidence set under standard covariate shift
                sset, _ = scs.get_confidence_set(Xtrain_nxp, ytrain_n, Xtest_1xp, lmbda, alpha=alpha) 
                sset_s.append(sset)
                scov_s[seed] = cal.is_covered(ytest_s[seed], sset, y_increment)

                if (seed + 1) % 100 == 0:
                    print("{}, {}, {}. {} trials. SCS, FCS coverage: {:.4f}, {:.4f}. {:.1f} s".format(
                        fitness_str, n_train, lmbda, seed + 1,
                        np.mean(scov_s[: seed + 1]), np.mean(fcov_s[: seed + 1]), time.time() - t0))

            np.savez('../results/{}_n{}_lambda{}_alpha{}_gamma{}.npz'.format(
                fitness_str, n_train, lmbda, alpha, reg),
                     ytest_s=ytest_s, predtest_s=predtest_s,
                     fset_s=fset_s, fcov_s=fcov_s, sset_s=sset_s, scov_s=scov_s, 
                    )